In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: iitj_workspace1
Azure region: eastus2
Subscription id: a6dd3c75-3739-4404-a6fd-d49a8c3b1da2
Resource group: iitj_resource1


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-project-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment
from azureml.core.script_run_config import ScriptRunConfig
import os

### YOUR CODE HERE ###
# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.1, 0.5, 1.0, 1.5),
    "--max_iter" : choice(50, 100, 150, 200)
})

### YOUR CODE HERE ###
# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
env = Environment.get(workspace=ws, name='AzureML-Tutorial')

### YOUR CODE HERE ###
# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = os.getcwd(),
                      compute_target = compute_target,
                      script = 'train.py',
                      environment=env
)

### YOUR CODE HERE ###
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=20,
                                     max_concurrent_runs=5)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [21]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()

# Best hyperdrive run information
print("Best Run ID :", best_run_hyperdrive.id)
print("Best Run Metrics :", best_run_hyperdrive.get_metrics())

# Best hyperdrive run paramaters
best_C = best_run_hyperdrive.get_metrics()['Regularization Strength:']
best_max_iter = best_run_hyperdrive.get_metrics()['Max iterations:']

Best Run ID : HD_8541259d-caab-4a74-ac76-32b07477c339_6
Best Run Metrics : {'Regularization Strength:': 1.0, 'Max iterations:': 200, 'Accuracy': 0.911750424860403}


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [14]:
from train import clean_data
from azureml.core.dataset import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Default Datastore
datastore = ws.get_default_datastore()

### YOUR DATA OBJECT HERE ###
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)

# Split data into train and test sets.
train, test = train_test_split(df, test_size=0.25, stratify=df.y, random_state=100)

#Register Dataset in Workspace
train_dataset = Dataset.Tabular.register_pandas_dataframe(
          dataframe=df, 
          name='bankmarketing_train', 
          description='Bank marketing dataset',
          target=datastore
          )

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/73ec1304-6330-4772-bd9e-561535025b93/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [23]:
# Saving best hyperdrive model
from sklearn.linear_model import LogisticRegression

# Train logistic regression using best hyperdrive parameters
best_hyperdrive_model = LogisticRegression(C=best_C, max_iter=best_max_iter)\
                        .fit(train.drop(['y'], axis=1), train['y'])

# Save model
joblib.dump(best_hyperdrive_model, 'outputs/best_hyperdrive_model.pkl')

['outputs/best_hyperdrive_model.pkl']

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_hours=0.5,
    task='classification',
    primary_metric='accuracy',
    training_data=train_dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target)

In [16]:
# Submit your automl run
### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on udacity-project-cluster with default configuration
Running on remote compute: udacity-project-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-1,AutoML_44c2cbea-400a-46d0-8b62-75ce8c03d093,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [24]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()
print(best_automl_run)

joblib.dump(best_automl_model, 'outputs/best_automl_model.pkl')

Run(Experiment: udacity-project-1,
Id: AutoML_44c2cbea-400a-46d0-8b62-75ce8c03d093_32,
Type: azureml.scriptrun,
Status: Completed)


['outputs/best_automl_model.pkl']

In [5]:
# cluster cleanup
compute_target.delete()